In [ ]:
from elasticsearch import Elasticsearch
import requests
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Elasticsearch 호스트 및 포트 설정
host = '218.50.209.93'
port = 20395
scheme = 'http'
es = Elasticsearch([{'host': host, 'port': port, 'scheme': scheme}])

# 인덱스, 변수 이름 설정
index_name = 'index'
userid = ''  # userid를 받아야하는 경우 여기에 설정
uid = ''  # uid를 받아야하는 경우 여기에 설정

# uid 또는 userid에 따라 필드를 선택
if uid:
    field_name = "uid.keyword"
else:
    field_name = "userId.keyword"

# 집계 쿼리 생성
aggs_query = {
    "aggs": {
        "2": {
            "terms": {
                "field": "requestUrl.keyword",
                "order": {
                    "_count": "desc"
                },
                "size": 5
            },
            "aggs": {
                "3": {
                    "avg": {
                        "field": "pageStayTime"
                    }
                }
            }
        }
    },
    "size": 0,
    "script_fields": {},
    "stored_fields": ["*"],
    "runtime_mappings": {},
    "query": {
        "bool": {
            "must": [],
            "filter": [
                {
                    "match_phrase": {
                        field_name: uid or userid
                    }
                },
                {
                    "match_phrase": {
                        "contentType.keyword": "bbs_policy_info"
                    }
                },
                {
                    "range": {
                        "requestTime": {
                            "format": "strict_date_optional_time",
                            "gte": "2023-08-01T00:00:00.000Z",
                            "lte": "2023-09-21T23:59:59.999Z"
                        }
                    }
                }
            ],
            "should": [],
            "must_not": []
        }
    }
}

# Elasticsearch에 집계 쿼리를 실행하고 결과를 받기
response = es.search(index=index_name, body=aggs_query)
# 결과 처리
aggregations = response.get('aggregations', {})
# URL, count, pageStaytime을 저장할 배열
url_data = []
selected_url_list = []
# Elasticsearch 집계 결과를 파싱하여 배열에 데이터 추가
buckets = aggregations.get("2", {}).get("buckets", [])
for bucket in buckets:
    url = bucket.get("key", "")
    count = bucket.get("doc_count", 0)
    page_stay_time = bucket.get("3", {}).get("value", 0)  # 페이지 체류 시간
    selected_url_list.append(url)
    # URL, count, pageStaytime을 딕셔너리로 묶어 배열에 추가
    url_data.append({
        "URL": url,
        "Count": count,
        "PageStayTime": page_stay_time
    })

# 클릭 수와 체류 시간 데이터
clicks = [data['Count'] for data in url_data]
dwell_times = [data['PageStayTime'] for data in url_data]

# 클릭 수와 체류 시간의 합을 계산
total_clicks = sum(clicks)
total_dwell_time = sum(dwell_times)

# 클릭 수와 체류 시간의 상대적 비율 계산
relative_clicks = [click / total_clicks for click in clicks]
relative_dwell_times = [dwell_time / total_dwell_time for dwell_time in dwell_times]

# 상대적 비율을 종합 점수로 계산 (예: 클릭 수와 체류 시간을 6:4 비율로 고려)
weighted_scores = [(0.6 * relative_clicks[i]) + (0.4 * relative_dwell_times[i]) for i in range(len(clicks))]

# 점수를 기준으로 데이터를 정렬하여 순위 지정
ranked_indices = sorted(range(len(weighted_scores)), key=lambda i: weighted_scores[i], reverse=True)


def send_to_engine_keyword(title, content):
    D_ENGINE_URL = "url"
    try:
        payload = {
            "title": title,
            "content": content
        }
        headers = {'Content-Type': 'application/json; charset=utf-8'}
        res = requests.post(D_ENGINE_URL, headers=headers, json=payload, verify=False)
        return res.json()
    except Exception as e:
        return {'error': str(e)}


def send_to_engine_embedding(title, content):
    D_ENGINE_URL = "url"
    try:
        payload = {
            "title": title,
            "content": content
        }
        headers = {'Content-Type': 'application/json; charset=utf-8'}
        res = requests.post(D_ENGINE_URL, headers=headers, json=payload, verify=False)
        return res.json()
    except Exception as e:
        return {'error': str(e)}


def jaccard_similarity(s1, s2):
    s1 = set(s1)
    s2 = set(s2)
    return len(s1 & s2) / len(s1 | s2)
# Elasticsearch 연결 생성
es = Elasticsearch([{'host': host, 'port': port, 'scheme': scheme}])

# Elasticsearch에서 데이터 가져오기
query = {"query": {"match_all": {}}}
result = es.search(index='index', size=10000)
hits_total = result['hits']['hits']

# Elasticsearch에서 가져온 데이터를 리스트에 저장
document_vectors = []
document_keywords = []
total_rank_list=[]
for hit in hits_total:
    source = hit['_source']
    vector = source.get('vector', '')
    keyword = source.get('keyword', '')
    url = source.get('url', '')
    document_vectors.append({"vector" : vector, "url" : url})
    document_keywords.append({"keyword" : keyword, "url" : url})

# 결과 출력
for i, idx in enumerate(ranked_indices):
    # print(f"Rank {i + 1}:")
    # print(f"  URL: {url_data[idx]['URL']}")
    # print(f"  Clicks: {clicks[idx]}")
    # print(f"  Dwell Time: {dwell_times[idx]} ms")
    # print(f"  Score: {weighted_scores[idx]}\n")

    # Elasticsearch에서 데이터 가져오기
    selected_url = url_data[idx]['URL']
    query = {
        "aggs": {},
        "size": 1,
        "script_fields": {},
        "stored_fields": ["_source"],
        "runtime_mappings": {},
        "query": {
            "bool": {
                "must": [
                    {
                        "match_phrase": {
                            "url": selected_url
                        }
                    }
                ],
                "should": [],
                "must_not": []
            }
        }
    }

    result = es.search(index='index-policy-2023', body=query)
    hits = result['hits']['hits']
    total_rank = []
    for hit in hits:
        source = hit['_source']
        url = source.get('url', '')
        keyword = source.get('keyword', '')
        title = source.get('title', '')
        content = source.get('content', '')
        vector = source.get('vector', '')

        # 입력 데이터 설정
        input_keyword_data = send_to_engine_keyword(title, content)
        input_embedding_data = send_to_engine_embedding(title, content)
        input_keyword_str = input_keyword_data['keywords']
        input_embedding_str = input_embedding_data['embedding']
        input_keywords = set(input_keyword_str)
        input_vector = np.array(input_embedding_str)

        # 자카드 유사도 계산
        similarities_jac = []
        for document_keyword in document_keywords:
            keyword = document_keyword['keyword']  # 'keyword'가 없으면 None을 반환
            if keyword is not None:
                similarity_jac = jaccard_similarity(input_keywords, keyword)
                similarities_jac.append((document_keyword['url'], similarity_jac))  # 'url'과 유사도를 튜플로 저장
            
        # 코사인 유사도 계산
        document_vectors_np = np.array([document_vector['vector'] for document_vector in document_vectors])
        similarities_cos = cosine_similarity([input_vector], document_vectors_np)
        most_similar_indices = np.argsort(similarities_cos[0])[::-1]  # 내림차순 정렬된 인덱스 배열
        # 상위의 유사한 데이터 출력
        top_5_results_cos = []
        for index in most_similar_indices[:50]:
            most_similar_source = hits_total[index]['_source']
            most_similar_url = most_similar_source['url']
            cosine_similarity_score = similarities_cos[0][index]
            top_5_results_cos.append((most_similar_url, cosine_similarity_score))
        # 자카드 유사도가 높은 순으로 결과 정렬
        sorted_results_jac = sorted(similarities_jac, key=lambda x: x[1], reverse=True)
        # 상위의 결과 선택
        top_5_results_jac = sorted_results_jac[:50]
        # top_5_results_cos와 top_5_results_jac에서 URL이 일치하는 값들의 점수 합산
        # combined_scores 리스트에 jac + cos score + weighted_scores[idx]를 저장
        combined_scores = []
        for url_cos, score_cos in top_5_results_cos:
            url_already_added = False  # URL이 이미 추가되었는지 확인하는 플래그
            for url_jac, score_jac in top_5_results_jac:
                if url_cos == url_jac:
                    total_score = score_cos + score_jac + weighted_scores[idx]
                    combined_scores.append((url_cos, total_score))
                    url_already_added = True  # URL이 추가되었음을 표시
                    break  # URL이 일치하면 반복문 종료
            # URL이 이미 추가되지 않았다면 score_cos만 추가
            if not url_already_added:
                total_score = score_cos + weighted_scores[idx]
                combined_scores.append((url_cos, total_score))
        # 합산 점수를 기준으로 정렬
        combined_scores = sorted(combined_scores, key=lambda x: x[1], reverse=True)
        # 결과 출력
        for i, (url, total_score) in enumerate(combined_scores[:5]):
            total_rank.append((url, total_score))
        total_rank_list.append(total_rank)
# total_rank_list를 점수(score)를 기준으로 정렬
total_rank_list = sorted(total_rank_list, key=lambda x: x[0][1], reverse=True)

# total_rank_list 내의 모든 URL과 점수를 하나의 리스트에 모음
all_scores = []
for ranked_items in total_rank_list:
    all_scores.extend(ranked_items)

# 스코어(score)를 기준으로 정렬
sorted_scores = sorted(all_scores, key=lambda x: x[1], reverse=True)
# 정렬된 스코어를 출력
final_set = []
for rank, (url, score) in enumerate(sorted_scores):
    if url not in selected_url_list:
        final_set.append((url, score))
        
for rank, (url, score) in enumerate(final_set[:5]):        
    print(f"Rank {rank + 1}:")
    print(f"  URL: {url}")
    print(f"  Score: {score}")
    print("-------------------------")

